## 1. Setup & Import Libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Paths
RAW_DATA_DIR = Path('../data/raw')
PROCESSED_DATA_DIR = Path('../data/processed')
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Setup complete!")


✅ Setup complete!


## 2. Load Raw Data

In [2]:
# Load all available parquet files
parquet_files = sorted(RAW_DATA_DIR.glob('*.parquet'))
print(f"📁 Found {len(parquet_files)} files:")
for f in parquet_files:
    print(f"   - {f.name}")


📁 Found 5 files:
   - yellow_tripdata_2024-01.parquet
   - yellow_tripdata_2024-02.parquet
   - yellow_tripdata_2024-03.parquet
   - yellow_tripdata_2024-04.parquet
   - yellow_tripdata_2024-05.parquet


In [3]:
# Load data - untuk efisiensi, kita bisa load beberapa bulan saja
# Atau gunakan sampling untuk development

# Option 1: Load 1 bulan saja (lebih cepat untuk development)
# df = pd.read_parquet(RAW_DATA_DIR / 'yellow_tripdata_2024-01.parquet')

# Option 2: Load semua data
dfs = []
for f in parquet_files:
    print(f"Loading {f.name}...")
    dfs.append(pd.read_parquet(f))

df = pd.concat(dfs, ignore_index=True)
print(f"\n✅ Total loaded: {len(df):,} rows")

# Free memory
del dfs


Loading yellow_tripdata_2024-01.parquet...
Loading yellow_tripdata_2024-02.parquet...
Loading yellow_tripdata_2024-03.parquet...
Loading yellow_tripdata_2024-04.parquet...
Loading yellow_tripdata_2024-05.parquet...

✅ Total loaded: 16,792,900 rows


In [4]:
# Quick overview
print(f"📊 Dataset Shape: {df.shape}")
print(f"\n📋 Columns: {list(df.columns)}")
df.head()


📊 Dataset Shape: (16792900, 19)

📋 Columns: ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee']


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


## 3. Data Cleaning

In [5]:
def clean_data(df):
    """
    Clean NYC Taxi data by removing invalid/anomalous records.
    """
    print("🧹 Starting Data Cleaning...")
    print(f"   Initial rows: {len(df):,}")
    
    # Make a copy
    df = df.copy()
    
    # 1. Remove rows with missing critical values
    critical_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 
                     'trip_distance', 'fare_amount', 'PULocationID', 'DOLocationID']
    initial = len(df)
    df = df.dropna(subset=critical_cols)
    print(f"   After removing missing critical values: {len(df):,} (removed {initial - len(df):,})")
    
    # 2. Fill missing passenger_count with mode (1)
    df['passenger_count'] = df['passenger_count'].fillna(1)
    
    # 3. Filter valid fare_amount ($2.50 minimum fare, max $500)
    initial = len(df)
    df = df[(df['fare_amount'] >= 2.5) & (df['fare_amount'] <= 500)]
    print(f"   After filtering fare ($2.5-$500): {len(df):,} (removed {initial - len(df):,})")
    
    # 4. Filter valid trip_distance (0.1 - 100 miles)
    initial = len(df)
    df = df[(df['trip_distance'] >= 0.1) & (df['trip_distance'] <= 100)]
    print(f"   After filtering distance (0.1-100 mi): {len(df):,} (removed {initial - len(df):,})")
    
    # 5. Filter valid passenger_count (1-6)
    initial = len(df)
    df = df[(df['passenger_count'] >= 1) & (df['passenger_count'] <= 6)]
    print(f"   After filtering passengers (1-6): {len(df):,} (removed {initial - len(df):,})")
    
    # 6. Filter valid total_amount
    initial = len(df)
    df = df[(df['total_amount'] >= 2.5) & (df['total_amount'] <= 1000)]
    print(f"   After filtering total ($2.5-$1000): {len(df):,} (removed {initial - len(df):,})")
    
    # 7. Create trip duration and filter
    df['trip_duration_minutes'] = (
        df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    ).dt.total_seconds() / 60
    
    initial = len(df)
    df = df[(df['trip_duration_minutes'] >= 1) & (df['trip_duration_minutes'] <= 180)]
    print(f"   After filtering duration (1-180 min): {len(df):,} (removed {initial - len(df):,})")
    
    # 8. Calculate and filter speed (remove impossible values)
    df['avg_speed_mph'] = df['trip_distance'] / (df['trip_duration_minutes'] / 60)
    initial = len(df)
    df = df[(df['avg_speed_mph'] >= 1) & (df['avg_speed_mph'] <= 70)]
    print(f"   After filtering speed (1-70 mph): {len(df):,} (removed {initial - len(df):,})")
    
    print(f"\n✅ Cleaning complete! Final rows: {len(df):,}")
    print(f"   Data retained: {len(df)/len(df)*100:.1f}%")
    
    return df.reset_index(drop=True)


In [6]:
# Clean the data
df_clean = clean_data(df)

# Free memory
del df


🧹 Starting Data Cleaning...
   Initial rows: 16,792,900
   After removing missing critical values: 16,792,900 (removed 0)
   After filtering fare ($2.5-$500): 16,498,086 (removed 294,814)
   After filtering distance (0.1-100 mi): 16,166,439 (removed 331,647)
   After filtering passengers (1-6): 15,988,325 (removed 178,114)
   After filtering total ($2.5-$1000): 15,988,323 (removed 2)
   After filtering duration (1-180 min): 15,961,008 (removed 27,315)
   After filtering speed (1-70 mph): 15,954,639 (removed 6,369)

✅ Cleaning complete! Final rows: 15,954,639
   Data retained: 100.0%


In [7]:
# Verify cleaning
print("📊 Data After Cleaning:")
print(f"   Shape: {df_clean.shape}")
print(f"\n   fare_amount range: ${df_clean['fare_amount'].min():.2f} - ${df_clean['fare_amount'].max():.2f}")
print(f"   trip_distance range: {df_clean['trip_distance'].min():.2f} - {df_clean['trip_distance'].max():.2f} miles")
print(f"   trip_duration range: {df_clean['trip_duration_minutes'].min():.1f} - {df_clean['trip_duration_minutes'].max():.1f} min")


📊 Data After Cleaning:
   Shape: (15954639, 21)

   fare_amount range: $2.50 - $500.00
   trip_distance range: 0.10 - 99.77 miles
   trip_duration range: 1.0 - 179.8 min


## 4. Feature Engineering

In [8]:
def engineer_features(df):
    """
    Create new features from raw data.
    """
    print("🔧 Starting Feature Engineering...")
    
    df = df.copy()
    
    # ===== TIME FEATURES =====
    print("   Creating time features...")
    
    # Basic time features
    df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
    df['pickup_day'] = df['tpep_pickup_datetime'].dt.day
    df['pickup_dayofweek'] = df['tpep_pickup_datetime'].dt.dayofweek
    df['pickup_month'] = df['tpep_pickup_datetime'].dt.month
    df['pickup_year'] = df['tpep_pickup_datetime'].dt.year
    
    # Is weekend (Saturday=5, Sunday=6)
    df['is_weekend'] = (df['pickup_dayofweek'] >= 5).astype(int)
    
    # Time of day categories
    def get_time_of_day(hour):
        if 6 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 17:
            return 'afternoon'
        elif 17 <= hour < 21:
            return 'evening'
        else:
            return 'night'
    
    df['time_of_day'] = df['pickup_hour'].apply(get_time_of_day)
    
    # Rush hour indicator
    df['is_rush_hour'] = ((
        ((df['pickup_hour'] >= 7) & (df['pickup_hour'] <= 9)) |  # Morning rush
        ((df['pickup_hour'] >= 16) & (df['pickup_hour'] <= 19))   # Evening rush
    ) & (df['pickup_dayofweek'] < 5)).astype(int)  # Only weekdays
    
    # ===== CYCLIC ENCODING =====
    print("   Creating cyclic features...")
    
    # Hour (24-hour cycle)
    df['hour_sin'] = np.sin(2 * np.pi * df['pickup_hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['pickup_hour'] / 24)
    
    # Day of week (7-day cycle)
    df['dow_sin'] = np.sin(2 * np.pi * df['pickup_dayofweek'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['pickup_dayofweek'] / 7)
    
    # Month (12-month cycle)
    df['month_sin'] = np.sin(2 * np.pi * df['pickup_month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['pickup_month'] / 12)
    
    # ===== TRIP FEATURES =====
    print("   Creating trip features...")
    
    # Speed already calculated in cleaning
    # But ensure it exists
    if 'avg_speed_mph' not in df.columns:
        df['avg_speed_mph'] = df['trip_distance'] / (df['trip_duration_minutes'] / 60)
    
    # Trip distance categories
    df['distance_category'] = pd.cut(
        df['trip_distance'],
        bins=[0, 1, 3, 5, 10, 100],
        labels=['very_short', 'short', 'medium', 'long', 'very_long']
    )
    
    # ===== LOCATION FEATURES =====
    print("   Creating location features...")
    
    # Same pickup and dropoff location
    df['same_location'] = (df['PULocationID'] == df['DOLocationID']).astype(int)
    
    # ===== FARE COMPONENTS =====
    print("   Creating fare features...")
    
    # Tip percentage (only for credit card payments)
    df['tip_percentage'] = np.where(
        df['fare_amount'] > 0,
        df['tip_amount'] / df['fare_amount'] * 100,
        0
    )
    
    # Has tip
    df['has_tip'] = (df['tip_amount'] > 0).astype(int)
    
    # Has tolls
    df['has_tolls'] = (df['tolls_amount'] > 0).astype(int)
    
    print(f"\n✅ Feature Engineering complete!")
    print(f"   Total features: {len(df.columns)}")
    
    return df


In [9]:
# Apply feature engineering
df_features = engineer_features(df_clean)

# Free memory
del df_clean


🔧 Starting Feature Engineering...
   Creating time features...
   Creating cyclic features...
   Creating trip features...
   Creating location features...
   Creating fare features...

✅ Feature Engineering complete!
   Total features: 40


In [10]:
# View new features
print("📊 All Features:")
for i, col in enumerate(df_features.columns, 1):
    print(f"   {i:2}. {col}")


📊 All Features:
    1. VendorID
    2. tpep_pickup_datetime
    3. tpep_dropoff_datetime
    4. passenger_count
    5. trip_distance
    6. RatecodeID
    7. store_and_fwd_flag
    8. PULocationID
    9. DOLocationID
   10. payment_type
   11. fare_amount
   12. extra
   13. mta_tax
   14. tip_amount
   15. tolls_amount
   16. improvement_surcharge
   17. total_amount
   18. congestion_surcharge
   19. Airport_fee
   20. trip_duration_minutes
   21. avg_speed_mph
   22. pickup_hour
   23. pickup_day
   24. pickup_dayofweek
   25. pickup_month
   26. pickup_year
   27. is_weekend
   28. time_of_day
   29. is_rush_hour
   30. hour_sin
   31. hour_cos
   32. dow_sin
   33. dow_cos
   34. month_sin
   35. month_cos
   36. distance_category
   37. same_location
   38. tip_percentage
   39. has_tip
   40. has_tolls


In [11]:
# Sample of engineered features
feature_cols = ['pickup_hour', 'pickup_dayofweek', 'is_weekend', 'is_rush_hour',
                'hour_sin', 'hour_cos', 'trip_duration_minutes', 'avg_speed_mph']
df_features[feature_cols].head(10)


,pickup_hour,pickup_dayofweek,is_weekend,is_rush_hour,hour_sin,hour_cos,trip_duration_minutes,avg_speed_mph
0,0,0,0,0,0.0,1.0,19.800000,5.212121
1,0,0,0,0,0.0,1.0,6.600000,16.363636
2,0,0,0,0,0.0,1.0,17.916667,15.739535
3,0,0,0,0,0.0,1.0,8.300000,10.120482
4,0,0,0,0,0.0,1.0,6.100000,7.868852
5,0,0,0,0,0.0,1.0,32.383333,8.708183
6,0,0,0,0,0.0,1.0,26.050000,24.921305
7,0,0,0,0,0.0,1.0,28.183333,11.581313
8,0,0,0,0,0.0,1.0,6.316667,7.124011
9,0,0,0,0,0.0,1.0,9.050000,7.955801


## 5. Select Features for Modeling

In [12]:
# Define features for modeling
# We'll use a mix of original and engineered features

NUMERICAL_FEATURES = [
    'trip_distance',
    'passenger_count',
    'trip_duration_minutes',
    'avg_speed_mph',
    'pickup_hour',
    'pickup_dayofweek',
    'pickup_month',
    'hour_sin',
    'hour_cos',
    'dow_sin',
    'dow_cos',
]

CATEGORICAL_FEATURES = [
    'PULocationID',
    'DOLocationID',
    'VendorID',
]

BINARY_FEATURES = [
    'is_weekend',
    'is_rush_hour',
    'same_location',
    'has_tolls',
]

TARGET = 'fare_amount'

# All features
ALL_FEATURES = NUMERICAL_FEATURES + CATEGORICAL_FEATURES + BINARY_FEATURES

print(f"📊 Features Selected:")
print(f"   Numerical: {len(NUMERICAL_FEATURES)}")
print(f"   Categorical: {len(CATEGORICAL_FEATURES)}")
print(f"   Binary: {len(BINARY_FEATURES)}")
print(f"   Total: {len(ALL_FEATURES)}")
print(f"   Target: {TARGET}")


📊 Features Selected:
   Numerical: 11
   Categorical: 3
   Binary: 4
   Total: 18
   Target: fare_amount


In [13]:
# Prepare features and target
X = df_features[ALL_FEATURES].copy()
y = df_features[TARGET].copy()

print(f"✅ Features shape: {X.shape}")
print(f"   Target shape: {y.shape}")


✅ Features shape: (15954639, 18)
   Target shape: (15954639,)


In [14]:
# Check for any remaining missing values
missing = X.isnull().sum()
if missing.sum() > 0:
    print("⚠️ Missing values found:")
    print(missing[missing > 0])
else:
    print("✅ No missing values in features")


✅ No missing values in features


## 6. Train/Validation/Test Split

In [15]:
# Split: 70% train, 15% validation, 15% test
RANDOM_STATE = 42

# First split: 70% train, 30% temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=RANDOM_STATE
)

# Second split: 50% validation, 50% test (from temp)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=RANDOM_STATE
)

print("📊 Data Split:")
print(f"   Train:      {len(X_train):>10,} ({len(X_train)/len(X)*100:.1f}%)")
print(f"   Validation: {len(X_val):>10,} ({len(X_val)/len(X)*100:.1f}%)")
print(f"   Test:       {len(X_test):>10,} ({len(X_test)/len(X)*100:.1f}%)")
print(f"   Total:      {len(X):>10,}")


📊 Data Split:
   Train:      11,168,247 (70.0%)
   Validation:  2,393,196 (15.0%)
   Test:        2,393,196 (15.0%)
   Total:      15,954,639


In [16]:
# Verify target distribution across splits
print("📊 Target Distribution:")
print(f"\n   Train:")
print(f"      Mean: ${y_train.mean():.2f}")
print(f"      Std:  ${y_train.std():.2f}")
print(f"\n   Validation:")
print(f"      Mean: ${y_val.mean():.2f}")
print(f"      Std:  ${y_val.std():.2f}")
print(f"\n   Test:")
print(f"      Mean: ${y_test.mean():.2f}")
print(f"      Std:  ${y_test.std():.2f}")


📊 Target Distribution:

   Train:
      Mean: $19.21
      Std:  $16.97

   Validation:
      Mean: $19.20
      Std:  $17.01

   Test:
      Mean: $19.22
      Std:  $17.03


## 7. Save Processed Data

In [17]:
# Combine X and y for saving
train_df = X_train.copy()
train_df[TARGET] = y_train.values

val_df = X_val.copy()
val_df[TARGET] = y_val.values

test_df = X_test.copy()
test_df[TARGET] = y_test.values

# Save to parquet
train_df.to_parquet(PROCESSED_DATA_DIR / 'train.parquet', index=False)
val_df.to_parquet(PROCESSED_DATA_DIR / 'val.parquet', index=False)
test_df.to_parquet(PROCESSED_DATA_DIR / 'test.parquet', index=False)

print("✅ Saved processed data:")
print(f"   {PROCESSED_DATA_DIR / 'train.parquet'}")
print(f"   {PROCESSED_DATA_DIR / 'val.parquet'}")
print(f"   {PROCESSED_DATA_DIR / 'test.parquet'}")


✅ Saved processed data:
   ../data/processed/train.parquet
   ../data/processed/val.parquet
   ../data/processed/test.parquet


In [ ]:
train_df.head()


,trip_distance,passenger_count,trip_duration_minutes,avg_speed_mph,pickup_hour,pickup_dayofweek,pickup_month,hour_sin,hour_cos,dow_sin,dow_cos,PULocationID,DOLocationID,VendorID,is_weekend,is_rush_hour,same_location,has_tolls,fare_amount
4490586,1.90,2.0,9.250000,12.324324,21,6,2,-0.707107,0.707107,-0.781831,0.62349,158,13,2,1,0,0,0,12.10
9780111,2.21,1.0,23.800000,5.571429,11,0,4,0.258819,-0.965926,0.000000,1.00000,100,234,2,0,0,0,0,21.20
11322373,2.60,2.0,15.100000,10.331126,21,1,4,-0.707107,0.707107,0.781831,0.62349,48,249,1,0,0,0,0,15.60
12167561,2.82,1.0,17.733333,9.541353,20,1,4,-0.866025,0.500000,0.781831,0.62349,170,236,2,0,0,0,0,19.44
15104445,2.47,1.0,8.633333,17.166023,23,6,5,-0.258819,0.965926,-0.781831,0.62349,141,74,2,1,0,0,0,12.10


: 

In [18]:
# Save feature configuration
feature_config = {
    'numerical_features': NUMERICAL_FEATURES,
    'categorical_features': CATEGORICAL_FEATURES,
    'binary_features': BINARY_FEATURES,
    'all_features': ALL_FEATURES,
    'target': TARGET,
    'random_state': RANDOM_STATE
}

joblib.dump(feature_config, PROCESSED_DATA_DIR / 'feature_config.joblib')
print(f"✅ Saved feature config: {PROCESSED_DATA_DIR / 'feature_config.joblib'}")


✅ Saved feature config: ../data/processed/feature_config.joblib


In [19]:
# Verify saved files
print("\n📁 Processed Data Files:")
for f in PROCESSED_DATA_DIR.glob('*'):
    size_mb = f.stat().st_size / (1024 * 1024)
    print(f"   {f.name}: {size_mb:.2f} MB")



📁 Processed Data Files:
   test.parquet: 42.80 MB
   feature_config.joblib: 0.00 MB
   val.parquet: 42.79 MB
   train.parquet: 200.59 MB


## 8. Summary

In [20]:
print("="*70)
print("                    📋 PREPROCESSING SUMMARY")
print("="*70)

print(f"""
DATA CLEANING:
   ✅ Removed invalid fare amounts (< $2.5 or > $500)
   ✅ Removed invalid trip distances (< 0.1 or > 100 miles)
   ✅ Removed invalid passenger counts (< 1 or > 6)
   ✅ Removed invalid trip durations (< 1 or > 180 minutes)
   ✅ Removed unrealistic speeds (< 1 or > 70 mph)

FEATURE ENGINEERING:
   ✅ Time features: hour, day, dayofweek, month
   ✅ Cyclic encoding: sin/cos for hour, day, month
   ✅ Binary features: is_weekend, is_rush_hour
   ✅ Trip features: duration, speed
   ✅ Location features: same_location

DATA SPLIT:
   ✅ Train: 70%
   ✅ Validation: 15%
   ✅ Test: 15%

SAVED FILES:
   📁 {PROCESSED_DATA_DIR}/
      - train.parquet
      - val.parquet
      - test.parquet
      - feature_config.joblib
""")

print("="*70)
print("✅ Preprocessing Complete! Ready for modeling.")
print("   Next: 03_modeling.ipynb")
print("="*70)


                    📋 PREPROCESSING SUMMARY

DATA CLEANING:
   ✅ Removed invalid fare amounts (< $2.5 or > $500)
   ✅ Removed invalid trip distances (< 0.1 or > 100 miles)
   ✅ Removed invalid passenger counts (< 1 or > 6)
   ✅ Removed invalid trip durations (< 1 or > 180 minutes)
   ✅ Removed unrealistic speeds (< 1 or > 70 mph)

FEATURE ENGINEERING:
   ✅ Time features: hour, day, dayofweek, month
   ✅ Cyclic encoding: sin/cos for hour, day, month
   ✅ Binary features: is_weekend, is_rush_hour
   ✅ Trip features: duration, speed
   ✅ Location features: same_location

DATA SPLIT:
   ✅ Train: 70%
   ✅ Validation: 15%
   ✅ Test: 15%

SAVED FILES:
   📁 ../data/processed/
      - train.parquet
      - val.parquet
      - test.parquet
      - feature_config.joblib

✅ Preprocessing Complete! Ready for modeling.
   Next: 03_modeling.ipynb
